In [2]:
import pandas as pd

power = pd.read_csv(
    "data/기본/법정동별시간별전력사용량.csv",
    encoding="utf-8"
)

power.head()

,SIGUNGU_CD,BJDONG_CD,USE_YM,USE_HM,FDRCT_VLD_KWH
0,11650,10700,20220628,100,10782.0565
1,11650,10800,20220628,100,11394.8635
2,11650,10900,20220628,100,7273.9620
3,11740,10300,20220628,100,11008.8110
4,11710,11300,20220628,100,2905.1120


In [3]:
dong_code = pd.read_excel(
    "data/기본/법정동코드 조회자료.xls"
)

dong_code.head()

,법정동코드,법정동명,폐지구분
0,1100000000,서울특별시,현존
1,1111000000,서울특별시 종로구,현존
2,1111010100,서울특별시 종로구 청운동,현존
3,1111010200,서울특별시 종로구 신교동,현존
4,1111010300,서울특별시 종로구 궁정동,현존


In [7]:
power.columns = power.columns.str.strip()

power["SIGUNGU_CD"] = power["SIGUNGU_CD"].astype(str).str.zfill(5)
power["BJDONG_CD"] = power["BJDONG_CD"].astype(str).str.zfill(5)

In [9]:
power["법정동코드"] = power["SIGUNGU_CD"] + power["BJDONG_CD"]

In [11]:
dong_code.columns = dong_code.columns.str.strip()
dong_code["법정동코드"] = dong_code["법정동코드"].astype(str).str.zfill(10)

power_merged = power.merge(
    dong_code[["법정동코드", "법정동명", "폐지구분"]],
    on="법정동코드",
    how="left"
)

In [12]:
power_merged.head(5)

,SIGUNGU_CD,BJDONG_CD,USE_YM,USE_HM,FDRCT_VLD_KWH,법정동코드,법정동명,폐지구분
0,11650,10700,20220628,100,10782.0565,1165010700,서울특별시 서초구 반포동,현존
1,11650,10800,20220628,100,11394.8635,1165010800,서울특별시 서초구 서초동,현존
2,11650,10900,20220628,100,7273.9620,1165010900,서울특별시 서초구 내곡동,현존
3,11740,10300,20220628,100,11008.8110,1174010300,서울특별시 강동구 상일동,현존
4,11710,11300,20220628,100,2905.1120,1171011300,서울특별시 송파구 거여동,현존


In [13]:
power_merged = power_merged.drop(columns=["SIGUNGU_CD", "BJDONG_CD"])

In [14]:
power_merged.head(5)

,USE_YM,USE_HM,FDRCT_VLD_KWH,법정동코드,법정동명,폐지구분
0,20220628,100,10782.0565,1165010700,서울특별시 서초구 반포동,현존
1,20220628,100,11394.8635,1165010800,서울특별시 서초구 서초동,현존
2,20220628,100,7273.9620,1165010900,서울특별시 서초구 내곡동,현존
3,20220628,100,11008.8110,1174010300,서울특별시 강동구 상일동,현존
4,20220628,100,2905.1120,1171011300,서울특별시 송파구 거여동,현존


In [15]:
new_cols = [
    # 지역
    "법정동코드",
    "법정동명",
    "폐지구분",
    "USE_YM",
    "USE_HM",
    "FDRCT_VLD_KWH"
]

power_merged = power_merged[new_cols]


In [16]:
power_merged.head(5)

,법정동코드,법정동명,폐지구분,USE_YM,USE_HM,FDRCT_VLD_KWH
0,1165010700,서울특별시 서초구 반포동,현존,20220628,100,10782.0565
1,1165010800,서울특별시 서초구 서초동,현존,20220628,100,11394.8635
2,1165010900,서울특별시 서초구 내곡동,현존,20220628,100,7273.9620
3,1174010300,서울특별시 강동구 상일동,현존,20220628,100,11008.8110
4,1171011300,서울특별시 송파구 거여동,현존,20220628,100,2905.1120


In [25]:
# 문자열 변환 (강제)
power_merged["USE_YM"] = power_merged["USE_YM"].astype(str)
power_merged["USE_HM"] = power_merged["USE_HM"].astype(str)

# 합친 문자열
dt_str = power_merged["USE_YM"] + power_merged["USE_HM"]

# 정상은 길이 12 (YYYYMMDDHHMM)
bad_rows = dt_str[dt_str.str.len() != 12]

bad_rows.head(10)

Series([], dtype: object)

In [26]:
power_merged["datetime"] = pd.to_datetime(
    {
        "year":  power_merged["USE_YM"].astype(str).str[:4],
        "month": power_merged["USE_YM"].astype(str).str[4:6],
        "day":   power_merged["USE_YM"].astype(str).str[6:8],
        "hour":  pd.to_numeric(power_merged["USE_HM"], errors="coerce") // 100,
        "minute": 0,
    }
)

In [28]:
power_merged[["USE_YM", "USE_HM", "datetime"]].head(10)


,USE_YM,USE_HM,datetime
0,20220628,0100,2022-06-28 01:00:00
1,20220628,0100,2022-06-28 01:00:00
2,20220628,0100,2022-06-28 01:00:00
3,20220628,0100,2022-06-28 01:00:00
4,20220628,0100,2022-06-28 01:00:00
5,20220628,0100,2022-06-28 01:00:00
6,20220628,0100,2022-06-28 01:00:00
7,20220628,0100,2022-06-28 01:00:00
8,20220628,0100,2022-06-28 01:00:00
9,20220628,0100,2022-06-28 01:00:00


In [29]:
power_merged = power_merged.drop(columns=["USE_YM", "USE_HM"])

In [30]:
power_merged.head(10)

,법정동코드,법정동명,폐지구분,FDRCT_VLD_KWH,datetime
0,1165010700,서울특별시 서초구 반포동,현존,10782.0565,2022-06-28 01:00:00
1,1165010800,서울특별시 서초구 서초동,현존,11394.8635,2022-06-28 01:00:00
2,1165010900,서울특별시 서초구 내곡동,현존,7273.9620,2022-06-28 01:00:00
3,1174010300,서울특별시 강동구 상일동,현존,11008.8110,2022-06-28 01:00:00
4,1171011300,서울특별시 송파구 거여동,현존,2905.1120,2022-06-28 01:00:00
5,1171011400,서울특별시 송파구 마천동,현존,3416.9310,2022-06-28 01:00:00
6,1174010100,서울특별시 강동구 명일동,현존,10473.6130,2022-06-28 01:00:00
7,1171010900,서울특별시 송파구 장지동,현존,14196.5290,2022-06-28 01:00:00
8,1171011100,서울특별시 송파구 방이동,현존,5894.3225,2022-06-28 01:00:00
9,1171011200,서울특별시 송파구 오금동,현존,6393.6870,2022-06-28 01:00:00


In [31]:
new_cols = [
    # 지역
    "법정동코드",
    "법정동명",
    "폐지구분",
    "datetime",
    "FDRCT_VLD_KWH"
]

power_merged = power_merged[new_cols]

In [32]:
power_merged.to_csv(
    "power_region.csv.gz",
    index=False,
    encoding="utf-8",
    compression="gzip"
)